In [1]:
!pip install aiokafka==0.8.1 pydantic==1.10.9 httpx==0.24.1 vcrpy==4.3.1 tenacity==8.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 337.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 757.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 726.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 446.8 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 729.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 1.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 624.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 680.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 579.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.4 MB/s eta 0:00:00a 0:00:01


In [2]:
from typing import Final

# https://tenders.guru/pl/api
BASE_URL: Final[str] = "https://tenders.guru/api/pl/tenders"

In [3]:
from pydantic import BaseModel, Field
import datetime

class TendersQueryParams(BaseModel):
    page: int

class TenderDetail(BaseModel):
    id: int
    date: datetime.date
    title: str = ""
    description: str = ""

class TendersListResponse(BaseModel):
    page_count: int
    page_number: int
    page_size: int
    total: int
    data: list[TenderDetail] = Field(default_factory=list)

In [4]:
import httpx
import tenacity

@tenacity.retry(
    wait=tenacity.wait_exponential(multiplier=1, min=4, max=10),
    stop=tenacity.stop_after_attempt(5),
)
async def get_tenders_list(client_: httpx.AsyncClient, page: int) -> TendersListResponse:
    response = await client_.get(
        BASE_URL,
        params=TendersQueryParams(page=page).dict(),
    )
    response.raise_for_status()
    return TendersListResponse.parse_obj(response.json())

In [5]:
import httpx

timeout = httpx.Timeout(10.0, connect=60.0)
limits = httpx.Limits(max_keepalive_connections=5, max_connections=10)
client = httpx.AsyncClient(timeout=timeout, limits=limits)

In [6]:
await get_tenders_list(client, 1)

TendersListResponse(page_count=5841, page_number=1, page_size=100, total=584070, data=[TenderDetail(id=586940, date=datetime.date(2023, 5, 3), title='Dostawa odczynników i testów. Część 1-5', description='Przedmiotem zamówienia jest: dostawa odczynników i testów w projekcie pt.: „Wdrożenie innowacyjnych elementów technologicznych w procesie wylęgu kaczek w celu ograniczenia zagrożeń mikrobiologicznych i poprawy jakości zdrowotnej i dobrostanu lężonych piskląt” z podziałem na części:\nCzęść nr 1: VA2 grupa 1a odczynniki do badań molekularnych\nCzęść nr 2: VA2 grupa 1b odczynniki do badań molekularnych – startery\nCzęść nr 3: VA2 grupa 2 odczynniki do elektroforezy\nCzęść nr 4: VA2 grupa 3 testy lekoop'), TenderDetail(id=586948, date=datetime.date(2023, 5, 3), title='Dostawa licencji IBM wraz z 12 miesięcznym wsparciem technicznym producenta', description='1. Przedmiotem zamówienia jest dostawa licencji wraz z zapewnieniem 12 miesięcznego wsparcia technicznego Producenta dla:\n1.1. IBM D

In [9]:
import vcr
import httpx
from aiokafka import AIOKafkaProducer


async def iterate_over_tenders():
    page = 1
    async with httpx.AsyncClient(timeout=timeout, limits=limits):
        while True:
            try:
                tenders: TendersListResponse = await get_tenders_list(client, page)
            except httpx.HTTPStatusError as e:
                if e.response.status_code == 404:
                    break
                else:
                    raise
            if not (tenders.page_size and tenders.data):
                break
            for tender in tenders.data:
                yield tender
            page += 1

def serialize_tender(tender: TenderDetail) -> bytes:
    return tender.json().encode("utf-8")

async def produce_tenders():
    producer = AIOKafkaProducer(
        bootstrap_servers="broker:9092",
        value_serializer=serialize_tender,
    )
    await producer.start()
    try:
        async for tender in iterate_over_tenders():
            await producer.send_and_wait("test", tender)
    finally:
        await producer.stop()

with vcr.use_cassette("work/tenders.yaml", record_mode="new_episodes"):
    await produce_tenders()

RetryError: RetryError[<Future at 0x7fdedf7826b0 state=finished raised HTTPStatusError>]

In [8]:
from aiokafka import AIOKafkaConsumer

async def consume_tenders():
    consumer = AIOKafkaConsumer(
        "test",
        bootstrap_servers="broker:9092",
        value_deserializer=TenderDetail.parse_raw,
        auto_offset_reset="earliest",
    )
    await consumer.start()
    try:
        async for tender in consumer:
            print(tender.value)
    finally:
        await consumer.stop()


await consume_tenders()

id=586940 date=datetime.date(2023, 5, 3) title='Dostawa odczynników i testów. Część 1-5' description='Przedmiotem zamówienia jest: dostawa odczynników i testów w projekcie pt.: „Wdrożenie innowacyjnych elementów technologicznych w procesie wylęgu kaczek w celu ograniczenia zagrożeń mikrobiologicznych i poprawy jakości zdrowotnej i dobrostanu lężonych piskląt” z podziałem na części:\nCzęść nr 1: VA2 grupa 1a odczynniki do badań molekularnych\nCzęść nr 2: VA2 grupa 1b odczynniki do badań molekularnych – startery\nCzęść nr 3: VA2 grupa 2 odczynniki do elektroforezy\nCzęść nr 4: VA2 grupa 3 testy lekoop'
id=586948 date=datetime.date(2023, 5, 3) title='Dostawa licencji IBM wraz z 12 miesięcznym wsparciem technicznym producenta' description='1. Przedmiotem zamówienia jest dostawa licencji wraz z zapewnieniem 12 miesięcznego wsparcia technicznego Producenta dla:\n1.1. IBM Db2 Standard Edition Cartridge for IBM Cloud Pak for Data Virtual Processor Core License + SW Subscription &amp; Support 1

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



CancelledError: 